In [1]:
from pathlib import Path

import numba as nb
import numpy as np
import scipy.stats as stats

from natsort import natsorted

import matplotlib.pyplot as plt


from threeML import *

#plt.style.use("thomas")
import warnings
warnings.simplefilter("ignore")


%matplotlib widget



Welcome to JupyROOT 6.22/02
[WARNING ] The naima package is not available. Models that depend on it will not be available
[WARNING ] The GSL library or the pygsl wrapper cannot be loaded. Models that depend on it will not be available.
[WARNING ] The ebltable package is not available. Models that depend on it will not be available
[INFO    ] Starting 3ML!
[WARNING ] Multinest minimizer not available
[WARNING ] PyGMO is not available
[WARNING ] The cthreeML package is not installed. You will not be able to use plugins which require the C/C++ interface (currently HAWC)
[WARNING ] Could not import plugin HAWCLike.py. Do you have the relative instrument software installed and configured?
[WARNING ] Could not import plugin FermiLATLike.py. Do you have the relative instrument software installed and configured?
[WARNING ] Env. variable OMP_NUM_THREADS is not set. Please set it to 1 for optimal performances in 3ML
[WARNING ] Env. variable MKL_NUM_THREADS is not set. Please set it to 1 for opti

In [2]:
spi = OGIPLike("spi",
               observation="fit_GRB120711A_BB_SwiftPosition_spec/spectra_GRB_OGIP.fits",
               response="fit_GRB120711A_BB_SwiftPosition_spec/spectral_response.rmf_OGIP.fits")



[WARNING ] Found TSTOP and TELAPSE. This file is invalid. Using TSTOP.
[WARNING ] FILTER is not set. This is not a compliant OGIP file. Assuming no FILTER.
[WARNING ] The response is in an extension called MATRIX, which usually means you also need an ancillary file (ARF) which you didn't provide. You should refer to the documentation  of the instrument and make sure you don't need an ARF.
[INFO    ] Auto-probed noise models:
[INFO    ] - observation: gaussian
[INFO    ] - background: None


In [3]:
spi.view_count_spectrum();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
band = Band()
band.alpha.prior = Truncated_gaussian(lower_bound=-1.5, upper_bound=1, mu=-1, sigma=0.5)
band.beta.prior = Truncated_gaussian(lower_bound=-5, upper_bound=-1.6, mu=-2, sigma=0.5)
band.xp.prior = Log_normal(mu=2, sigma=1)
band.xp.bounds = (0, None)
band.K.prior = Log_uniform_prior(lower_bound=1e-10, upper_bound=1e3)
ps = PointSource("grb", 0, 0, spectral_shape=band)
band_model = Model(ps)

In [5]:
bayes = BayesianAnalysis(band_model, DataList(spi))

In [6]:
bayes.set_sampler()
bayes.sampler.setup(n_live_points=400,n_iterations=2000)
_ = bayes.sample()

[INFO    ] using default sampler emcee
[INFO    ] sampler is setup with default parameters


  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

[INFO    ] Mean acceptance fraction: 0.5819750000000001
Maximum a posteriori probability (MAP) point:



,result,unit
parameter,,
grb.spectrum.main.Band.K,(3.02 +/- 0.12) x 10^-2,1 / (cm2 keV s)
grb.spectrum.main.Band.alpha,(-9.1 +/- 0.5) x 10^-1,
grb.spectrum.main.Band.xp,(1.31 +/- 0.25) x 10^3,keV
grb.spectrum.main.Band.beta,-2.5 +/- 0.5,



Values of -log(posterior) at the minimum:



,-log(posterior)
spi,-20.667021
total,-20.667021



Values of statistical measures:



,statistical measures
AIC,50.815523
BIC,55.196985
DIC,49.062371
PDIC,3.764031


In [7]:
bayes.restore_median_fit()

display_spectrum_model_counts(bayes);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
plot_spectra(bayes.results, flux_unit="erg2/(cm2 s keV)");

processing MLE analyses: 0it [00:00, ?it/s]

processing Bayesian analyses:   0%|          | 0/1 [00:00<?, ?it/s]

Propagating errors:   0%|          | 0/100 [00:00<?, ?it/s]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
bayes.results.corner_plot();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …